In [1]:
import csv
import pandas as pd
import numpy as np

# Чтение данных

In [2]:
news_train = csv.reader(open('news_train.txt', 'rt', encoding="utf8"), delimiter='\t')
df = pd.DataFrame(news_train, columns = ['Genres', 'Title', 'Publication'])
df.head()

,Genres,Title,Publication
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г..."
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс..."


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 3 columns):
Genres         60000 non-null object
Title          60000 non-null object
Publication    60000 non-null object
dtypes: object(3)
memory usage: 1.4+ MB


# Очистка и преобразование данных

In [4]:
import nltk
import pymorphy2
import re
nltk.download('stopwords')
from nltk.corpus import stopwords # Import the stop word list

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kiril\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
def review_to_words(review_text):
    # Function to convert a raw review to a string of words
    morph = pymorphy2.MorphAnalyzer()
    # 1. Remove non-letters        
    letters_only = re.sub("[^а-яА-Яa-zA-Z]", " ", review_text) 
    #
    # 2. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 3. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("russian"))                  
    # 
    # 4. Remove stop words and transform to normal_form
    meaningful_words = [morph.parse(w)[0].normal_form for w in words if not w in stops]   
    #
    # 5. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join(meaningful_words)) 

In [6]:
df['text'] = df['Title'] + ' ' + df['Publication']

In [29]:
%%time
df['Clear_text'] = df['text'].apply(review_to_words)

Wall time: 1h 15min 39s


In [32]:
df.head()

,Genres,Title,Publication,text,Clear_text
0,style,Rolex наградит победителей регаты,Парусная гонка Giraglia Rolex Cup пройдет в Ср...,Rolex наградит победителей регаты Парусная гон...,rolex наградить победитель регата парусный гон...
1,sport,Матс Сундин стал советником тренера сборной Шв...,Шведский хоккеист Матс Сундин назначен советни...,Матс Сундин стал советником тренера сборной Шв...,матс сундина стать советник тренер сборный шве...
2,media,Брендом года по версии EFFIE впервые стал город,"Гран-при конкурса ""Брэнд года/EFFIE"" получил г...",Брендом года по версии EFFIE впервые стал горо...,бренд год версия effie впервые стать город гра...
3,economics,Цена нефти WTI снизилась после публикации данн...,Цена американской нефти WTI на лондонской бирж...,Цена нефти WTI снизилась после публикации данн...,цена нефть wti снизиться публикация дать запас...
4,economics,Сбербанк распродаст другим банкирам миллиардны...,"Сбербанк выставил на продажу долги по 21,4 тыс...",Сбербанк распродаст другим банкирам миллиардны...,сбербанк распродать другой банкир миллиард дол...


In [7]:
#df.to_csv('cleared_train_data.csv', index=False)
#df = pd.read_csv('cleared_train_data.csv')

# Обучение модели

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Исппользован мешок слов

In [14]:
vectorizer = CountVectorizer(analyzer = "word",  
                                 tokenizer = None,
                                 preprocessor = None, 
                                 stop_words = None)

### Использован TF-IDF

In [28]:
vectorizer = TfidfVectorizer(analyzer = "word",  
                                 tokenizer = None,
                                 preprocessor = None, 
                                 stop_words = None)

Преобразование данных

In [29]:
%%time
train_data_features = vectorizer.fit_transform(df['Clear_text']) 

Wall time: 9.71 s


Кроссвалидация модели и обучение модели

In [30]:
%%time
clf = LogisticRegression(max_iter=1000)
clf_params = {'C':[15, 20, 30, 40]}
clf_grid = GridSearchCV(clf, clf_params, cv=3)
clf_grid.fit(train_data_features, df['Genres'])

Wall time: 39min 25s


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [15, 20, 30, 40]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [31]:
clf_grid.best_score_

0.8973833333333333

In [32]:
clf_grid.best_params_

{'C': 20}

In [33]:
%%time
clf = LogisticRegression(max_iter=1000, dual=True, solver='liblinear')
clf_params = {'C':[15, 20, 30, 40]}
clf_grid_lin = GridSearchCV(clf, clf_params, cv=3)
clf_grid_lin.fit(train_data_features, df['Genres'])

Wall time: 3min 48s


GridSearchCV(cv=3, error_score=nan,
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=True,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=1000, multi_class='auto',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='liblinear',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='deprecated', n_jobs=None, param_grid={'C': [15, 20, 30, 40]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [34]:
clf_grid_lin.best_score_

0.8978

In [35]:
clf_grid_lin.best_params_

{'C': 30}

In [36]:
if clf_grid.best_score_ > clf_grid_lin.best_score_:
    clf_grid_best = clf_grid
else:
    clf_grid_best = clf_grid_lin

# Предсказания для тестовой выборки

In [76]:
news_test = csv.reader(open('news_test.txt', 'rt', encoding="utf8"), delimiter='\t')
df_test = pd.DataFrame(news_test, columns = ['Title', 'Publication'])

In [77]:
df_test['text'] =  df_test['Title'] + ' ' + df_test['Publication']

In [121]:
df_test.head()

,Title,Publication,text,Clear_text
0,В МИД Белоруссии одобрили вынос российского фл...,"Белорусский спортсмен, несший на открытии пара...",В МИД Белоруссии одобрили вынос российского фл...,мид белоруссия одобрить вынос российский флаг ...
1,Про Хана Соло и Бобу Фетта из «Звездных войн» ...,"Стало известно, кто именно будет центральными ...",Про Хана Соло и Бобу Фетта из «Звездных войн» ...,хан соло боб фетта звёздный война снять отдель...
2,В Рунете началась вирусная ICQ-эпидемия,Тысячи пользователей стали жертвой нового ICQ-...,В Рунете началась вирусная ICQ-эпидемия Тысячи...,рунет начаться вирусный icq эпидемия тысяча по...
3,Шуховскую башню на Оке признали памятником фед...,Башня конструкции инженера Владимира Шухова в ...,Шуховскую башню на Оке признали памятником фед...,шуховсковать башня ока признать памятник федер...
4,Голодец предупредила правительство о последств...,Вице-премьер правительства России Ольга Голоде...,Голодец предупредила правительство о последств...,голодец предупредить правительство последствие...


In [83]:
%%time
df_test['Clear_text'] = df_test['text'].apply(review_to_words)

Wall time: 18min 33s


In [37]:
#df_test.to_csv('cleared_test_data.csv', index=False)
df_test = pd.read_csv('cleared_test_data.csv')

In [40]:
%%time
test_data_features = vectorizer.transform(df_test['Clear_text']) 

Wall time: 2.34 s


In [41]:
prediction = clf_grid_best.best_estimator_.predict(test_data_features)
prediction

array(['sport', 'culture', 'media', ..., 'economics', 'science', 'travel'],
      dtype=object)

In [42]:
with open("submission_new.txt",'w') as text:
    for word in prediction:
        text.write(word + '\n')

In [43]:
pd.Series(prediction).value_counts()

culture      2207
economics    2198
science      2130
sport        2130
media        2096
life         2070
forces       1216
business      414
style         271
travel        268
dtype: int64

In [44]:
df['Genres'].value_counts()

science      8657
media        8629
economics    8545
sport        8510
culture      8405
life         8083
forces       4758
business     2099
travel       1166
style        1148
Name: Genres, dtype: int64